In [ ]:
from hw2.training import ClassifierTrainer
from hw2.answers import part2_optim_hp
from itertools import product
from tqdm.auto import tqdm

torch.manual_seed(seed)

# Define a tiny part of the CIFAR-10 dataset to overfit it
batch_size = 2
max_batches = 25
dl_train = torch.utils.data.DataLoader(ds_train, batch_size, shuffle=False)

# Create model, loss and optimizer instances
model = ArgMaxClassifier(
    model=CNN(
        in_size, num_classes, channels=[32], pool_every=1, hidden_dims=[100],
        conv_params=dict(kernel_size=3, stride=1, padding=1),
        pooling_params=dict(kernel_size=2),
    )
)

# lr = [1e-1,5e-1,1e-2,5e-2,
#       1e-1,5e-1,1e-2,5e-2,
#       1e-3,5e-3,1e-4,5e-4,
#       1e-5,5e-5]
# weight_decay = [0.1,0.01,0.05,0.001]
# momentum = [0.6,0.7,0.8,0.85,0.9,0.95,0.99] 

hp_optim = part2_optim_hp()
loss_fn = hp_optim.pop('loss_fn')
lr = [2e-4]
weight_decay = [0.01]
momentum = [0.98] 


hyper_params = product(enumerate(lr), enumerate(weight_decay),enumerate(momentum))
accumulator_best_acc = []
    
for (i, lr), (j, weight_decay), (q,momentum) in tqdm(list(hyper_params)):
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)

    # Use ClassifierTrainer to run only the training loop a few times.
    trainer = ClassifierTrainer(model, loss_fn, optimizer, device)
    best_acc = 0


    for i in range(25):
        res = trainer.train_epoch(dl_train, max_batches=max_batches, verbose=(i%5==0))
        best_acc = res.accuracy if res.accuracy > best_acc else best_acc
    accumulator_best_acc.append(best_acc)
    print(accumulator_best_acc)
    print(f'{lr=},{weight_decay=},{momentum=},{best_acc=}')
    if(best_acc > 90 ):
        print('Found')
        break
    # # Test overfitting
    # test.assertGreaterEqual(best_acc, 90)
